In [2]:
!pip install pytesseract
!pip install easyocr

In [5]:
import cv2
import pytesseract
import re
import os

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\Tesseract-OCR\\tesseract.exe'

video_path = 'asdasd.mp4'

cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_skip = max(total_frames // 10, 1)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('output_video.avi', fourcc, 30, (1280, 720))

text_saved = [] 

if not os.path.exists("carplates"):
    os.makedirs("carplates")

for frame_index in range(0, total_frames, frame_skip):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    
    if not ret:
        break
    
    height, width = frame.shape[:2]
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(blurred, 120, 255, 1)
    
    cnts = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    for i, c in enumerate(cnts):
        x, y, w, h = cv2.boundingRect(c)
        if w >= 3 * h and w <= 6 * h and w <= width / 3 and w >= width / 15:
            config = "--psm 11 --oem 3 -c textord_tabfind_find_tables=0 textord_tabfind_only_strokewidths_below_height_fraction=0"
            roi = gray[y:y+h, x:x+w]
            text = pytesseract.image_to_string(roi, lang='eng', config=config)
            
            text = re.sub(r'\s+', '', text)
            text = re.sub(r'\W', '', text)
            
            if len(text) > 4:
                print(f"Detected text: {text}")
                text_saved.append(text)  # Add detected text to the list
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), (36, 255, 12), 2)
                cropped_region = frame[y:y+h, x:x+w]
                cv2.imwrite(f"carplates/frame_{frame_index}_region_{i}.jpg", cropped_region)
    
    output_video.write(frame)
    
text_saved = list(set(text_saved))

cap.release()
output_video.release()

print(f"All detected texts: {text_saved}")

<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\daneker\ipykernel_16208\2986747757.py:6: SyntaxWarning: invalid escape sequence '\T'
  pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\Tesseract-OCR\\tesseract.exe'


Detected text: 990H7404
Detected text: meFYUHTA07EIHzA04
Detected text: oeAeta__ifseea
Detected text: ii8ss_re
Detected text: 90HzA01
Detected text: ireree
Detected text: 990HZA01
Detected text: _eeel_
Detected text: m2990HZA101
Detected text: SEEEE
Detected text: aeesi
Detected text: M29SOHZA
All detected texts: ['oeAeta__ifseea', '_eeel_', '990HZA01', 'ii8ss_re', '90HzA01', 'aeesi', 'm2990HZA101', 'M29SOHZA', 'SEEEE', 'meFYUHTA07EIHzA04', '990H7404', 'ireree']


In [5]:

directory = "carplates"

reader = easyocr.Reader(['en'])

for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        file_path = os.path.join(directory, filename)
        result = reader.readtext(file_path)
        if result:
            print(f"Text in {filename}:")
            for detection in result:
                print(detection[1]) 
            print("----")
        else:
            print(f"No text detected in {filename}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


No text detected in frame_28_region_1.jpg
No text detected in frame_35_region_2.jpg
Text in frame_432_region_55.jpg:
9OHZA
----
No text detected in frame_56_region_1.jpg
